In [5]:
import pandas as pd
df = pd.read_excel('/Users/dejmen/desktop/ironhack/week2/day1/GSAF5.xls')
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-06-17 00:00:00,2025.0,Unprovoked,USA,South Carolina,Hilton Head Island,Swimming,Not stated,F,12,...,Not stated,Kevin McMurray Trackingsharks.com:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-11 00:00:00,2025.0,Unprovoked,USA,Florida,Boca Grande,Snorkeling,Leah Lendel,F,9,...,Bull shark,Kevin McMurray Trackingsharks.com: James Kings...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-05-29 00:00:00,2025.0,Unprovoked,USA,North Carolina,Sunset Beach,Swimming,Sean Barton,M,26,...,Not stated,Kevin McMurray Trackingsharks.com: Clay Crewel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-05-26 00:00:00,2025.0,Unprovoked,Vanuatu,South Santo,Espiitu Santo Island,Swimming,Tumas,M,14,...,Not stated,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-05-15 00:00:00,2025.0,Unprovoked,Australia,South Australia,Port Noarlunga,Swimming,Richard Vinall,M,66,...,Not stated,Simon DeMarchi: Todd Smith: 9 News:ABC News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = pd.read_excel('GSAF5.xls')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7020 entries, 0 to 7019
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            7020 non-null   object 
 1   Year            7018 non-null   float64
 2   Type            7002 non-null   object 
 3   Country         6970 non-null   object 
 4   State           6535 non-null   object 
 5   Location        6454 non-null   object 
 6   Activity        6435 non-null   object 
 7   Name            6801 non-null   object 
 8   Sex             6441 non-null   object 
 9   Age             4026 non-null   object 
 10  Injury          6985 non-null   object 
 11  Fatal Y/N       6459 non-null   object 
 12  Time            3494 non-null   object 
 13  Species         3889 non-null   object 
 14  Source          7001 non-null   object 
 15  pdf             6799 non-null   object 
 16  href formula    6794 non-null   object 
 17  href            6796 non-null   o

In [7]:
df["Type"].tail()

7015    Unprovoked
7016    Unprovoked
7017    Unprovoked
7018    Unprovoked
7019    Unprovoked
Name: Type, dtype: object

In [8]:
df["Injury"].isnull().sum()

np.int64(35)

In [9]:
df["Time"].isnull().sum()

np.int64(3526)

In [16]:
df["Date"].isnull

<bound method Series.isnull of 0       2025-06-17 00:00:00
1       2025-06-11 00:00:00
2       2025-05-29 00:00:00
3       2025-05-26 00:00:00
4       2025-05-15 00:00:00
               ...         
7015            Before 1903
7016            Before 1903
7017              1900-1905
7018              1883-1889
7019              1845-1853
Name: Date, Length: 7020, dtype: object>

In [50]:
import pandas as pd
import re

def clean_time(value):
    if pd.isna(value):
        return None

    value = str(value).strip().lower()

    # Remove useless or unclear values
    if value in ["?", "am", "pm", "unknown", "not stated", "n/a", "na"]:
        return None

    # Clean formats like "after 1200hr", "11.30hr", "15.5", etc.
    match = re.search(r'(\d{1,2})[h:.]?(\d{2})?', value)

    if match:
        hour = match.group(1)
        minute = match.group(2) if match.group(2) else "00"
        return f"{hour.zfill(2)}:{minute.zfill(2)}"

    # Keep known phrases like "Morning", "Afternoon", etc.
    return value.title()
from datetime import datetime



In [52]:
from datetime import datetime

def precise_time_to_day_part(value):
    if value is None:
        return None  # Keep missing as None

    # Known descriptive phrases to keep untouched
    descriptive_parts = [
        "Early Morning", "Morning", "Midday", "Early Afternoon",
        "Late Afternoon", "Afternoon", "Evening", "Dusk",
        "Night", "Late Night"
    ]
    
    if isinstance(value, str) and value.title() in descriptive_parts:
        return value.title()

    try:
        # Try parsing standard time like "14:30"
        time = datetime.strptime(value, "%H:%M").time()
        hour = time.hour
        minute = time.minute

        if 5 <= hour < 8:
            return "Early Morning"
        elif 8 <= hour < 12:
            return "Morning"
        elif hour == 12 and minute == 0:
            return "Midday"
        elif 12 <= hour < 15:
            return "Early Afternoon"
        elif 15 <= hour < 17:
            return "Late Afternoon"
        elif 17 <= hour < 19:
            return "Evening"
        elif 19 <= hour < 20:
            return "Dusk"
        elif 20 <= hour < 24:
            return "Night"
        else:  # 00:00 to before 5:00
            return "Late Night"
    except:
        return None  # Unrecognized values go to None


In [54]:
df['Cleaned_Time'] = df['Time'].apply(clean_time)
df['Day_Part'] = df['Cleaned_Time'].apply(precise_time_to_day_part)
print(df['Day_Part'].value_counts())


Day_Part
Morning            940
Early Afternoon    634
Late Afternoon     611
Evening            415
Afternoon          215
Early Morning      182
Midday             139
Night              125
Dusk                79
Late Night          38
Name: count, dtype: int64


In [56]:
df['Cleaned_Time'].head(60)

0     12:00
1     12:11
2     11:00
3      None
4     09:45
5      None
6     15:00
7      None
8      None
9     15:00
10    14:30
11    12:10
12    13:40
13     None
14    13:00
15    08:15
16    13:00
17    15:03
18    18:30
19    18:30
20     None
21    16:45
22    17:11
23    16:00
24    13:30
25    13:40
26    16:15
27    17:10
28     None
29     None
30     None
31    16:37
32     None
33     None
34     None
35     None
36     None
37     None
38    16:30
39     None
40     None
41     None
42     None
43     None
44    16:00
45     None
46    11:30
47    11:00
48    17:35
49    11:00
50    16:00
51     None
52     None
53     None
54    11:00
55    11:00
56    12:00
57    14:00
58    15:00
59    13:15
Name: Cleaned_Time, dtype: object

In [58]:
print(df['Day_Part'].value_counts(dropna=False))


Day_Part
None               3642
Morning             940
Early Afternoon     634
Late Afternoon      611
Evening             415
Afternoon           215
Early Morning       182
Midday              139
Night               125
Dusk                 79
Late Night           38
Name: count, dtype: int64


In [10]:
df["Date"].value_counts().tail(60)

Date
10-Mar-2000              1
14-Mar-2000              1
15-Mar-2000              1
24-Mar-2000              1
26-Mar-2000              1
30-Mar-2000              1
31-Mar-2000              1
09-Apr-2000              1
14-Apr-2000              1
09-May-2000              1
13-May-2000              1
Early Jun-2000           1
02-Jun-2000              1
10-Jun-2000              1
13-Jun-2000              1
19-Jun-2000              1
11-Sep-2000              1
15-Sep-2000              1
03-May-2001              1
03-Mar-2001              1
09-Jan-2001              1
21-Jan-2001              1
24-Jan-2001              1
Reported  24-Jan-2001    1
04-Feb-2001              1
11-Feb-2001              1
26-Feb-2001              1
Mar-2001                 1
08-Mar-2001              1
19-Sep-2000              1
09-Mar-2001              1
23-Mar-2001              1
02-Apr-2001              1
02-Ap-2001               1
05-Apr-2001              1
10-Apr-2001              1
28-Apr-2001            

In [18]:
import pandas as pd
import re


def clean_date(date):
    date = str(date)
    
    # Remove known unwanted words
    cleaned_date = re.sub(r'\b(Reported|Early|Before|No date|No Date)\b', '', date, flags=re.IGNORECASE)
    cleaned_date = re.sub(r'[^0-9a-zA-Z\-/ :]', '', cleaned_date).strip()

    # Try known formats first
    for fmt in ("%d-%b-%Y", "%d %b-%Y", "%Y-%m-%d %H:%M:%S", "%d-%m-%Y", "%Y-%m-%d"):
        try:
            return pd.to_datetime(cleaned_date, format=fmt, errors='raise')
        except:
            continue

    # Fall back to automatic parsing (dayfirst off for ISO formats)
    return pd.to_datetime(cleaned_date, errors='coerce', dayfirst=False)
df['Cleaned_Date'] = df['Date'].apply(clean_date)


In [22]:
df['Cleaned_Date'].tail(60)

6960          NaT
6961          NaT
6962   1960-01-01
6963   1959-06-19
6964   1959-04-24
6965   1958-01-01
6966   1958-01-01
6967   1958-01-01
6968   1958-01-01
6969   1958-01-01
6970   1958-01-01
6971   1957-01-01
6972   1957-01-01
6973   1956-01-01
6974   1956-01-01
6975   1956-03-01
6976   1952-01-01
6977          NaT
6978          NaT
6979          NaT
6980          NaT
6981          NaT
6982          NaT
6983          NaT
6984          NaT
6985          NaT
6986          NaT
6987          NaT
6988   1905-01-01
6989          NaT
6990          NaT
6991   1905-01-01
6992          NaT
6993          NaT
6994          NaT
6995   1927-01-01
6996          NaT
6997          NaT
6998          NaT
6999          NaT
7000          NaT
7001   1921-01-01
7002   1911-01-01
7003   1921-01-01
7004   1921-01-01
7005   1917-01-01
7006   1916-07-17
7007          NaT
7008   1913-07-19
7009   1911-01-01
7010          NaT
7011   1906-01-01
7012   1906-01-01
7013   1906-01-01
7014   1906-01-01
7015   190

In [24]:
def get_season(date):
    if pd.isna(date):
        return "No Date"
    
    month = date.month
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Autumn"
    elif month in [12, 1, 2]:
        return "Winter"
    
    return "No Date"

# Apply season mapping
df['Season'] = df['Cleaned_Date'].apply(get_season)

print(df[['Date', 'Cleaned_Date', 'Season']])

                     Date Cleaned_Date   Season
0     2025-06-17 00:00:00   2025-06-17   Summer
1     2025-06-11 00:00:00   2025-06-11   Summer
2     2025-05-29 00:00:00   2025-05-29   Spring
3     2025-05-26 00:00:00   2025-05-26   Spring
4     2025-05-15 00:00:00   2025-05-15   Spring
...                   ...          ...      ...
7015          Before 1903   1903-01-01   Winter
7016          Before 1903   1903-01-01   Winter
7017            1900-1905          NaT  No Date
7018            1883-1889          NaT  No Date
7019            1845-1853          NaT  No Date

[7020 rows x 3 columns]


In [79]:
df['Season'].value_counts()

Season
Summer     1979
Winter     1854
Autumn     1556
Spring     1410
No Date     221
Name: count, dtype: int64

In [62]:
# print(df["Injury"].value_counts().to_string())


In [60]:
counts = df["Injury"].value_counts()

# Filter where count is exactly 1
single_occurrences = counts[counts == 1]

# Show how many have count = 1
print(f"Number of unique injuries that appear only once: {len(single_occurrences)}")

Number of unique injuries that appear only once: 3708


In [64]:
df["Injury_clean"] = (
    df["Injury"]
    .str.lower()
    .str.strip()
    .str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-letter characters
)

# Quick grouping preview
print(df["Injury_clean"].value_counts())

Injury_clean
fatal                                                              882
foot bitten                                                        100
survived                                                            97
no injury                                                           94
leg bitten                                                          84
                                                                  ... 
punctures on left foot and foot                                      1
several  puncture wounds on lower right leg                          1
heel  foot bitten                                                    1
fatal lower thigh  knee severely lacerated                           1
fatal shark bit him in half carrying away the lower extremities      1
Name: count, Length: 4014, dtype: int64


In [66]:
def simplify_injury(text):
    if pd.isna(text):
        return "unknown"
    text = text.lower()
    if "fatal" in text:
        return "fatal"
    elif "foot" in text:
        return "foot injury"
    elif "leg" in text:
        return "leg injury"
    elif "hand" in text:
        return "hand injury"
    elif "no injury" in text:
        return "no injury"
    else:
        return "other"

df["Injury_grouped"] = df["Injury"].apply(simplify_injury)
print(df["Injury_grouped"].value_counts())

Injury_grouped
other          2481
fatal          1434
leg injury      902
foot injury     864
no injury       856
hand injury     448
unknown          35
Name: count, dtype: int64


In [71]:
print(df["Injury_grouped"].value_counts().to_string())


Injury_grouped
other          2481
fatal          1434
leg injury      902
foot injury     864
no injury       856
hand injury     448
unknown          35


In [11]:
                                                 # HIPOLITO PART

#CLEAN THE FATAL COlUMN:


# Check the unique values in the column 'Fatal Y/N':
print(df['Fatal Y/N'].dropna().apply(lambda x: repr(x)).unique())

["'N'" "'Y'" "'F'" "'M'" "'n'" "'Nq'" "'UNKNOWN'" '2017' "'Y x 2'" "' N'"
 "'N '" "'y'"]


In [12]:
#Change to string and eliminate spaces
df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()
#Check for valid values
valid_values = {'Y': 'Y', 'N': 'N'}
#Put all the good values the rest will be NaN
df['Fatal Y/N'] = df['Fatal Y/N'].map(valid_values)
#Result
print(df['Fatal Y/N'].unique())

['N' 'Y' nan]


In [13]:
# Count of values (Y, N)
print(df['Fatal Y/N'].value_counts(dropna=False))

Fatal Y/N
N      4897
Y      1480
NaN     643
Name: count, dtype: int64


In [14]:
# Check the unique values in the column 'Country'
print(df['Country'].unique())

['USA' 'Vanuatu' 'Australia' 'Jamaica' 'Israel' 'Mexico' 'Maldives'
 'Philippines' 'Bahamas' 'Turks and Caicos' 'Mozambique' 'New Caledonia'
 'Egypt' 'Thailand' 'New Zealand' 'Hawaii' 'Honduras' 'Indonesia'
 'Morocco' 'Belize' 'Maldive Islands' 'French Polynesia' 'Tobago'
 'AUSTRALIA' 'INDIA' 'TRINIDAD' 'BAHAMAS' 'SOUTH AFRICA' 'MEXICO'
 'NEW ZEALAND' 'EGYPT' 'BELIZE' 'PHILIPPINES' 'Coral Sea' 'SPAIN'
 'PORTUGAL' 'SAMOA' 'COLOMBIA' 'ECUADOR' 'FRENCH POLYNESIA'
 'NEW CALEDONIA' 'TURKS and CaICOS' 'CUBA' 'BRAZIL' 'SEYCHELLES'
 'ARGENTINA' 'FIJI' 'MeXICO' 'South Africa' 'ENGLAND' 'JAPAN' 'INDONESIA'
 'JAMAICA' 'MALDIVES' 'THAILAND' 'COLUMBIA' 'COSTA RICA'
 'British Overseas Territory' 'CANADA' 'JORDAN' 'ST KITTS / NEVIS'
 'ST MARTIN' 'PAPUA NEW GUINEA' 'REUNION ISLAND' 'ISRAEL' 'CHINA'
 'IRELAND' 'ITALY' 'MALAYSIA' 'LIBYA' nan 'MAURITIUS' 'SOLOMON ISLANDS'
 'ST HELENA, British overseas territory' 'COMOROS' 'REUNION'
 'UNITED KINGDOM' 'UNITED ARAB EMIRATES' 'CAPE VERDE' 'Fiji'
 'DOMINICAN 

In [15]:
#All in mayus and eliminate spaces
df['Country'] = df['Country'].str.strip().str.upper()
print(df['Country'].unique())


['USA' 'VANUATU' 'AUSTRALIA' 'JAMAICA' 'ISRAEL' 'MEXICO' 'MALDIVES'
 'PHILIPPINES' 'BAHAMAS' 'TURKS AND CAICOS' 'MOZAMBIQUE' 'NEW CALEDONIA'
 'EGYPT' 'THAILAND' 'NEW ZEALAND' 'HAWAII' 'HONDURAS' 'INDONESIA'
 'MOROCCO' 'BELIZE' 'MALDIVE ISLANDS' 'FRENCH POLYNESIA' 'TOBAGO' 'INDIA'
 'TRINIDAD' 'SOUTH AFRICA' 'CORAL SEA' 'SPAIN' 'PORTUGAL' 'SAMOA'
 'COLOMBIA' 'ECUADOR' 'CUBA' 'BRAZIL' 'SEYCHELLES' 'ARGENTINA' 'FIJI'
 'ENGLAND' 'JAPAN' 'COLUMBIA' 'COSTA RICA' 'BRITISH OVERSEAS TERRITORY'
 'CANADA' 'JORDAN' 'ST KITTS / NEVIS' 'ST MARTIN' 'PAPUA NEW GUINEA'
 'REUNION ISLAND' 'CHINA' 'IRELAND' 'ITALY' 'MALAYSIA' 'LIBYA' nan
 'MAURITIUS' 'SOLOMON ISLANDS' 'ST HELENA, BRITISH OVERSEAS TERRITORY'
 'COMOROS' 'REUNION' 'UNITED KINGDOM' 'UNITED ARAB EMIRATES' 'CAPE VERDE'
 'DOMINICAN REPUBLIC' 'CAYMAN ISLANDS' 'ARUBA' 'PUERTO RICO'
 'ATLANTIC OCEAN' 'GREECE' 'ST. MARTIN' 'FRANCE' 'TRINIDAD & TOBAGO'
 'KIRIBATI' 'DIEGO GARCIA' 'TAIWAN' 'PALESTINIAN TERRITORIES' 'GUAM'
 'NIGERIA' 'TONGA' 'SCOTLAND' '

In [16]:
# CLEANING DATA OF COUNTRIES

import numpy as np

country_corrections = {
    # Correccions Ortografics
    'COLUMBIA': 'COLOMBIA',
    'TRINIDAD & TOBAGO': 'TRINIDAD AND TOBAGO',
    'MALDIVE ISLANDS': 'MALDIVES',
    'UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES',
    'ST. MARTIN': 'ST MARTIN',
    'ST. MAARTIN': 'ST MARTIN',
    'TRINIDAD': 'TRINIDAD AND TOBAGO',

    # Agrupations
    'ENGLAND': 'UK',
    'SCOTLAND': 'UK',
    'UNITED KINGDOM': 'UK',
    'BRITISH ISLES': 'UK',
    'BRITISH WEST INDIES': 'UK',
    'BRITISH VIRGIN ISLANDS': 'UK',

    # Ocean y region not usefull
    'PACIFIC OCEAN': 'OTHER',
    'ATLANTIC OCEAN': 'OTHER',
    'INDIAN OCEAN': 'OTHER',
    'SOUTH PACIFIC OCEAN': 'OTHER',
    'CARIBBEAN SEA': 'OTHER',
    'OCEAN': 'OTHER',
    'GULF OF ADEN': 'OTHER',
    'MID-PACIFC OCEAN': 'OTHER',
    'NORTH ATLANTIC OCEAN': 'OTHER',
    'RED SEA': 'OTHER',
    'RED SEA / INDIAN OCEAN': 'OTHER',
    'NORTH PACIFIC OCEAN': 'OTHER',
    'CENTRAL PACIFIC': 'OTHER',

    # Some other mistakes → agrupar
    'DIEGO GARCIA': 'OTHER',
    'JOHNSTON ISLAND': 'OTHER',
    'ADMIRALTY ISLANDS': 'OTHER',
    'MID ATLANTIC OCEAN': 'OTHER',
    'UNKNOWN': 'OTHER',
    'AFRICA': 'OTHER',
    'ASIA?': 'OTHER',
    'SUDAN?': 'SUDAN',
}

In [17]:
# Import the country correction on or columns

def clean_column_country(df, column='Country'):
    # All mayus
    df[column] = df[column].str.strip().str.upper()
    #use the country_corrections to filter the column
    df[column] = df[column].replace(country_corrections)
    return df

df = clean_column_country(df, column='Country')

print(sorted(df['Country'].dropna().unique()))

['ALGERIA', 'AMERICAN SAMOA', 'ANDAMAN / NICOBAR ISLANDAS', 'ANDAMAN ISLANDS', 'ANGOLA', 'ANTIGUA', 'ARGENTINA', 'ARUBA', 'AUSTRALIA', 'AZORES', 'BAHAMAS', 'BAHREIN', 'BANGLADESH', 'BARBADOS', 'BAY OF BENGAL', 'BELIZE', 'BERMUDA', 'BETWEEN PORTUGAL & INDIA', 'BRAZIL', 'BRITISH NEW GUINEA', 'BRITISH OVERSEAS TERRITORY', 'BURMA', 'CANADA', 'CAPE VERDE', 'CAYMAN ISLANDS', 'CEYLON', 'CEYLON (SRI LANKA)', 'CHILE', 'CHINA', 'COAST OF AFRICA', 'COLOMBIA', 'COMOROS', 'COOK ISLANDS', 'CORAL SEA', 'COSTA RICA', 'CRETE', 'CROATIA', 'CUBA', 'CURACAO', 'CYPRUS', 'DJIBOUTI', 'DOMINICAN REPUBLIC', 'ECUADOR', 'EGYPT', 'EGYPT / ISRAEL', 'EL SALVADOR', 'EQUATORIAL GUINEA / CAMEROON', 'FALKLAND ISLANDS', 'FEDERATED STATES OF MICRONESIA', 'FIJI', 'FRANCE', 'FRENCH POLYNESIA', 'GABON', 'GEORGIA', 'GHANA', 'GRAND CAYMAN', 'GREECE', 'GREENLAND', 'GRENADA', 'GUAM', 'GUATEMALA', 'GUINEA', 'GUYANA', 'HAITI', 'HAWAII', 'HONDURAS', 'HONG KONG', 'ICELAND', 'INDIA', 'INDIAN OCEAN?', 'INDONESIA', 'IRAN', 'IRAN / IRA

In [18]:
# Top 10 countries with more sharks attacts:
top_10_paises = df['Country'].value_counts().head(10)
print(top_10_paises)

Country
USA                 2561
AUSTRALIA           1502
SOUTH AFRICA         598
NEW ZEALAND          146
BAHAMAS              139
PAPUA NEW GUINEA     136
BRAZIL               122
MEXICO               107
OTHER                 83
ITALY                 72
Name: count, dtype: int64


In [6]:
activity_counts = df['Activity'].value_counts()

# Get top 10
top_10 = activity_counts.head(10)

# Calculate sum of all other activities
other_count = activity_counts[10:].sum()

# Create a new series with "Other activities" included
top_10_with_other = pd.concat([
    top_10, 
    pd.Series({'Other activities': other_count})
])

print("Top 10 activities plus others:")
print(top_10_with_other)

print("\nFormatted table:")
print(top_10_with_other.to_markdown())

Top 10 activities plus others:
Surfing             1133
Swimming             997
Fishing              490
Spearfishing         388
Wading               177
Bathing              164
Diving               147
Snorkeling           132
Standing             113
Scuba diving          84
Other activities    2610
dtype: int64

Formatted table:
|                  |    0 |
|:-----------------|-----:|
| Surfing          | 1133 |
| Swimming         |  997 |
| Fishing          |  490 |
| Spearfishing     |  388 |
| Wading           |  177 |
| Bathing          |  164 |
| Diving           |  147 |
| Snorkeling       |  132 |
| Standing         |  113 |
| Scuba diving     |   84 |
| Other activities | 2610 |


In [7]:
import pandas as pd

def get_activity_stats(df, activity_column='Activity', top_n=10):

    # Make copy to avoid modifying original DataFrame
    df = df.copy()
    
    # Standard cleaning
    df[activity_column] = (
        df[activity_column]
        .str.strip()  # Remove whitespace
        .str.lower()  # Convert to lowercase
    )
    
    # Common activity replacements (customize as needed)
    activity_replacements = {
        'swim': 'swimming',
        'bike': 'cycling',
        'bicycle': 'cycling',
        'football': 'soccer',
        'bball': 'basketball',
        'hoops': 'basketball',
        # Add more as needed for your dataset
    }
    
    df[activity_column] = df[activity_column].replace(activity_replacements)
    
    # Get activity counts
    activity_counts = df[activity_column].value_counts().reset_index()
    activity_counts.columns = ['Activity', 'Count']
    total = activity_counts['Count'].sum()
    
    # Separate top N and others
    top_activities = activity_counts.head(top_n)
    other_count = activity_counts['Count'][top_n:].sum()
    
    # Create "Other" row
    other_row = pd.DataFrame({
        'Activity': ['Other activities'],
        'Count': [other_count],
        'Percentage': [(other_count / total * 100).round(1)]
    })
    
    # Calculate percentages for top activities
    top_activities['Percentage'] = (top_activities['Count'] / total * 100).round(1)
    
    # Combine results
    result = pd.concat([top_activities, other_row], ignore_index=True)
    
    return result

# Usage example:
activity_stats = get_activity_stats(df, top_n=10)

print("Activity Statistics (Top 10 + Others):")
print(activity_stats.to_markdown(index=False, floatfmt=".1f"))

Activity Statistics (Top 10 + Others):
| Activity         |   Count |   Percentage |
|:-----------------|--------:|-------------:|
| surfing          |    1138 |         17.7 |
| swimming         |    1044 |         16.2 |
| fishing          |     506 |          7.9 |
| spearfishing     |     396 |          6.2 |
| wading           |     177 |          2.8 |
| bathing          |     167 |          2.6 |
| diving           |     150 |          2.3 |
| snorkeling       |     133 |          2.1 |
| standing         |     115 |          1.8 |
| scuba diving     |     104 |          1.6 |
| Other activities |    2505 |         38.9 |


/var/folders/74/2xg39v_50tv7vj24jw0r0b800000gn/T/ipykernel_28325/3083507710.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_activities['Percentage'] = (top_activities['Count'] / total * 100).round(1)


In [8]:
def clean_sex_column(df, column_name='Sex'):
    """
    Cleans a sex column by:
    1. Converting all values to strings
    2. Stripping whitespace
    3. Converting to uppercase
    4. Keeping only 'M' or 'F'
    5. Setting all others to NaN
    """
    
    # Convert to string, strip whitespace, uppercase
    cleaned = df[column_name].astype(str).str.strip().str.upper()
    
    # Keep only M or F, others become NaN
    df[column_name] = cleaned.where(cleaned.isin(['M', 'F']))
    
    # Count cleaned values
    print("\nCleaning report:")
    print(f"Original value counts:\n{df[column_name].astype(str).str.strip().str.upper().value_counts()}")
    print(f"\nValid 'M' count: {(cleaned == 'M').sum()}")
    print(f"Valid 'F' count: {(cleaned == 'F').sum()}")
    print(f"Invalid values set to NaN: {len(df) - ((cleaned == 'M').sum() + (cleaned == 'F').sum())}")
    
    return df

df_cleaned = clean_sex_column(df, 'Sex')

print("\nFirst 10 rows after cleaning:")
print(df_cleaned[['Sex']].head(10))


Cleaning report:
Original value counts:
Sex
M      5634
F       802
NAN     584
Name: count, dtype: int64

Valid 'M' count: 5634
Valid 'F' count: 802
Invalid values set to NaN: 584

First 10 rows after cleaning:
  Sex
0   F
1   F
2   M
3   M
4   M
5   F
6   M
7   M
8   M
9   M
